# Kymograph generator from trench zarr

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm.notebook import tqdm
import zarr
import napari
import PIL.ImageDraw
import PIL.Image
import PIL.ImageFont
from skimage.io import imshow, imread
from joblib import Parallel, delayed
from math import ceil

In [ ]:
trenches = zarr.open("trenches.zarr", mode="r")
trenches.shape

In [ ]:
def create_timestamp_str_secs(time, imaging_interval=3):
    """
    Takes the time point as an integer (e.g. 0, 1, ..., 63), and converts it to an
    hh.mm.ss timestamp as a string. 
    Imaging interval supplied in seconds.
    """
    total_secs = time * imaging_interval
    hours = total_secs // 3600
    remainder_secs = total_secs % 3600
    remainder_mins = remainder_secs // 60
    just_secs = remainder_secs % 60
    timestamp = "{}.{}.{}".format(str(hours).zfill(2), str(remainder_mins).zfill(2), str(just_secs).zfill(2))
    
    return timestamp

In [ ]:
def get_trench(trench_zarr, trench_num, timepoint, channel, add_timestamp=False, font_size=10):
    """
    Pulls the trench array from the zarr, and adds a timestamp if selected
    """
    
    if add_timestamp:
        trench = trench_zarr[trench_num, timepoint, channel, :, :]
        I0 = PIL.Image.fromarray(trench, mode="I;16")
        I1 = PIL.ImageDraw.Draw(I0, mode="I;16")
        font = PIL.ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", font_size, encoding="unic")
        
        # make the pixel intensity of the timestamp appropriate
        img_intensity = np.mean(trench) # could also try max intensity if this performs poorly
        font_pixel_intensity_factor = 3  # changes the brightness of the timestamp, 3 is a good value
        fill = font_pixel_intensity_factor * (img_intensity/(2**16))*256
        
        I1.text((5, 5), create_timestamp_str_secs(timepoint), fill=ceil(fill), font=font)
        img = np.array(I0)
    else:
        img = trench_zarr[trench_num, timepoint, channel, :, :]
    
    return img

In [ ]:
def create_kymograph(trench_zarr, trench_num, time_range, channel, add_timestamp=False, font_size=10, save_image=False, kymo_path="kymographs/"):
    """
    Creates a kymograph from specified trenches.
    """
    trenches = [get_trench(trench_zarr, trench_num, time, channel, add_timestamp=add_timestamp, font_size=font_size) for time in time_range]
    kymo = np.concatenate(trenches, axis=1)
    
    if save_image:
        file_name = os.path.join(kymo_path, "kymo_TR{}_{}.png".format(str(trench_num).zfill(5), channel))
        im = PIL.Image.fromarray(kymo)
        im.save(file_name)
    
    return kymo

### Test kymograph from zarr array

In [ ]:
time_range = [100*x for x in [y for y in range(0,15)]]
time_range

In [ ]:
kymo = create_kymograph(trenches, 4, time_range, 2, add_timestamp=True, font_size=22, save_image=False)
fig, ax = plt.subplots(figsize=(40, 20))
ax.imshow(kymo)

### Generate kymographs

In [ ]:
try:
    os.mkdir("kymographs")
except:
    pass

In [ ]:
############################################

# Generate kymographs from png files

In [ ]:
import matplotlib.pyplot as plt
import tifffile
import numpy as np
import os
from glob import glob
from tqdm.notebook import tqdm
import PIL.ImageDraw
import PIL.Image
import PIL.ImageFont
from skimage.io import imshow, imread
from natsort import natsorted
from joblib import Parallel, delayed

In [ ]:
### TODO convert this to be in seconds

def create_timestamp_str(time, imaging_interval=2.5):
    """
    Takes the time point as an integer (e.g. 0, 1, ..., 63), and converts it to an
    hh.mm.ss timestamp as a string.
    """
    total_mins = time * imaging_interval
    hours = int(total_mins // 60)
    mins = total_mins % 60
    just_mins = int(mins - (mins % 1))
    secs = int((mins % 1) * 60)
    timestamp = "{}.{}.{}".format(str(hours).zfill(2), str(just_mins).zfill(2), str(secs).zfill(2))
    
    return timestamp

In [ ]:
def create_timestamp_str_secs(time, imaging_interval=150):
    """
    Takes the time point as an integer (e.g. 0, 1, ..., 63), and converts it to an
    hh.mm.ss timestamp as a string. 
    Imaging interval supplied in seconds.
    """
    total_secs = time * imaging_interval
    hours = total_secs // 3600
    remainder_secs = total_secs % 3600
    remainder_mins = remainder_secs // 60
    just_secs = remainder_secs % 60
    timestamp = "{}.{}.{}".format(str(hours).zfill(2), str(remainder_mins).zfill(2), str(just_secs).zfill(2))
    
    return timestamp

In [ ]:
def get_trench(trenches_path, FOV, channel, trench_num, time, file_ext="png", add_timestamp=False, font_size=10):
    """
    Creates a numpy array of a specified trench. Adding a timestamp is optional.
    
    Parameters
    ----------
    trenches_path : str
        The full path to the folder containing the trenches
    FOV : int
        The desired FOV
    channel : str
        The desired colour channel 
    trench_num : int
        The desired trench number
    time : int
        The time point of the desired trench
    file_ext : str
        The file extension of the trench file, png or tif
    add_timestamp: bool
        Whether or not to add a timestamp to the image.
        
    Returns
    -------
    trench: np.ndarray
        The numpy array of the specified trench.
    """
    trench_file = os.path.join(trenches_path, "xy{}_{}_TR{}_T{}.{}".format(str(FOV).zfill(3), channel, str(trench_num).zfill(2), str(time).zfill(4), file_ext))
    
    if add_timestamp:
        I0 = PIL.Image.open(trench_file)
        I1 = PIL.ImageDraw.Draw(I0)
        font = PIL.ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", font_size, encoding="unic")
        # make the pixel intensity of the timestamp appropriate
        if channel == "PC":
            fill = 40000
        if channel == "mCherry":
            fill = 800
        if channel == "YFP":
            fill = 1000
        if channel == "mVenus":
            fill = 2000
        if channel == "CFP":
            fill = 5
        I1.text((5, 5), create_timestamp_str(time), fill=fill, font=font)
        trench = np.array(I0)
    
    else:
        if file_ext == "tif":
            trench = tifffile.imread(trench_file)
        else:
            trench = imread(trench_file)
        
    return trench

In [ ]:
def create_kymograph(trenches_path, FOV, channel, trench_num, time_range, file_ext="png", timestamp=False, font_size=16, save_image=False):
    """
    Creates a kymograph from specified trenches.
    #TODO create ability to only put in, e.g. every second trench or timepoint
    #TODO create this as a function of a new class, trench
    #TODO for this new class, trench, we should create a natural indexing system.
    #TODO make the kymographs a 4 dimensional object, such that you can flick through the colour channels in fiji
    
    Parameters
    ----------
    trenches_path : str
        The full path to the folder containing the trenches
    FOV : int
        The desired FOV
    channel : str
        The desired colour channel 
    trench_num : int
        The desired trench number
    time_range : list
        The time points to use in the kymograph
        
    Returns
    -------
    kymo: numpy.ndarray
        The concatenated trenches forming a kymograph.
    """
    trenches = [get_trench(trenches_path, FOV, channel, trench_num, time, file_ext=file_ext, add_timestamp=timestamp, font_size=font_size) for time in time_range]
    kymo = np.concatenate(trenches, axis=1)
    
    if save_image:
        file_name = os.path.join(kymo_path, "kymo_xy{}_{}_TR{}.{}".format(str(FOV).zfill(3), channel, str(trench_num).zfill(2), file_ext))
        im = PIL.Image.fromarray(kymo)
        im.save(file_name)
    
    return kymo

### Test kymograph 
* Try testing in each channel to ensure the pixel intensity of the text is suitable

In [ ]:
trenches_path = os.getcwd() + "/trenches"
#trenches_path = trenches_path.encode('unicode-escape').decode()

In [ ]:
kymo = create_kymograph(trenches_path, 10, "mVenus", 1, [i for i in range(120,150)], timestamp=True, font_size=16)
fig, ax = plt.subplots(figsize=(40, 20))
ax.imshow(kymo)

### Create single channel kymographs

In [ ]:
try:
    os.mkdir("kymographs")
except:
    pass

In [ ]:
# useful parameters for making the kymographs
kymo_path = os.getcwd() + "/kymographs" 
trench_files = natsorted(glob(trenches_path + "/*.png"))
num_FOVs = 96
channel_list = ["mCherry", "PC", "mVenus"]
num_times = 181

# find the number of trenches in each FOV
num_trenches_per_FOV = []
for FOV in range(num_FOVs):
    files = [f for f in trench_files if ("xy{}_mCherry".format(str(FOV).zfill(3)) in f) and ("T0000" in f)]
    num_trenches_per_FOV.append(len(files))
print(num_trenches_per_FOV)
print(len(num_trenches_per_FOV))

Beware that this is a very memory intensive step to create kymographs in parallel, might require ~40G of memory across the RAM and swapfile.

In [ ]:
for FOV in tqdm(range(num_FOVs)):
    for ch in channel_list:
        trench_list = range(num_trenches_per_FOV[FOV])
        Parallel(n_jobs=-1)(delayed(create_kymograph)(trenches_path, FOV, ch, tr, [i for i in range(num_times)], timestamp=True, font_size=16, save_image=True) for tr in trench_list)      

### Create multi channel kymographs
* Allows colour overlays with independent colour channel adjustment in Fiji
* In principle, multi-channel kymographs could be stacked so you could access each trench and FOV with an independent slider

In [ ]:
def create_overlay_kymograph(kymo_path, FOV, channel_list, trench, input_file_ext="png", out_path=None, save_image=False):
    """
    Merge single channel kymographs into overlay kymographs. Best to specify colour channels in the channel_list 
    in the order red -> green -> blue -> gray, but it's ultimately arbitrary and doesn't really matter, any LUT can be applied in Fiji.
    """
    kymo_names = ["kymo_xy{}_{}_TR{}.{}".format(str(FOV).zfill(3), channel, str(trench).zfill(2), input_file_ext) for channel in channel_list]
    kymo_files = [os.path.join(kymo_path, f) for f in kymo_names]
    if input_file_ext == "png": 
        kymo_arrays = tuple([imread(f) for f in kymo_files])
    if input_file_ext == "tif":
        kymo_arrays = tuple([tifffile.imread(f) for f in kymo_files])
    else:
        assert input_file_ext == "png", "Check your input file extension type, should be png or tif"
    rgb = np.dstack(kymo_tuple)
    
    if save_image:
        outfile = os.path.join(out_path, "overlay_kymo_xy{}_TR{}.tif".format(str(FOV).zfill(3), str(trench).zfill(2)))
        tifffile.imwrite(outfile, rgb)
        
    return rgb

In [ ]:
#os.mkdir("kymographs_overlay")

In [ ]:
kymo_path = os.getcwd() + "/kymographs" 
out_path = os.getcwd() + "/kymographs_overlay"
ch_list = ["mCherry", "mVenus", "CFP", "PC"]
num_FOVs = 45

In [ ]:
for FOV in tqdm(range(num_FOVs)):
    trench_list = range(num_trenches_per_FOV[FOV])
    Parallel(n_jobs=-1)(delayed(create_overlay_kymograph)(kymo_path=kymo_path, 
                                                          FOV=FOV, 
                                                          channel_list=ch_list, 
                                                          trench=tr, 
                                                          input_file_ext="png", 
                                                          out_path=out_path, 
                                                          save_image=True) for tr in trench_list)